In [1]:
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill, Color, Border, Side

import warnings
warnings.filterwarnings('ignore')

1. title 시트에 title format 컬럼 생각해서 다시 만들어야 함


## 경로지정

In [2]:
raw_address = 'F:/OneDrive/3. 검수/1. 상반기 국가별 검수 Raw'
final_address = 'F:/OneDrive/3. 검수/Kia.com(국가) SEO Audit_200403.xlsx'
countrys = ['Pakistan(PK)']
as_countrys = ['pk']

## 1. internal_all 전처리 - 검수 URL 생성

In [3]:
# for country, as_country in zip(countrys, as_countrys):
#     # 데이터 불러오기
#     check_url = pd.read_excel(raw_address+'/'+country+'/Raw/internal_all.xlsx', sheet_name='1 - All')
    
#     # 데이터 가공
#     check_url_1 = check_url['Address'].str.contains('www.kia.com/'+as_country+'/')
#     response_url = check_url[check_url_1 == True]
#     response_url.sort_values(by=['Status Code'],ascending=False, inplace=True) # response 완료
    
    
#     checked_url_2 = response_url[response_url['Status Code'] != '404']

    
    
    
    
    
    
# #     Non_indexable_url = checked_url_2['Indexability'] == 'Non-Indexable'
# #     Non_indexable_url = checked_url_2[Non_indexable_url] # Non_indexable 완료
    
    

In [4]:
# # redirect 부분 편집해야 함

# address_url_2 = []
# address_url_3 = []
# address_url_4 = []
# address_url_5 = []

# if any(checked_url_2['Redirect URL'].notnull() == True):
#     redirect_url_1 = checked_url_2['Redirect URL'][checked_url_2['Redirect URL'].notnull() == True]
#     for url in redirect_url_1:
#         redirect_url_1_address = checked_url_2[url == checked_url_2['Address']]
#         if redirect_url_1_address['Status Code'].item() == 200:
#             print('ok')
#         else:
#             address_url_2.append(url)
#     if any(checked_url_2['Address'].isin(address_url_2) == True): 
#         redirect_url_2 = checked_url_2['Redirect URL'][checked_url_2['Address'].isin(address_url_2) == True]
#         for url in redirect_url_2:
#             redirect_url_2_address = checked_url_2[url == checked_url_2['Address']]
#             if redirect_url_2_address['Status Code'].item() == 200:
#                 print('ok')
#             else:
#                 address_url_3.append(url)
#         if any(checked_url_2['Address'].isin(address_url_3) == True): 
#             redirect_url_3 = checked_url_2['Redirect URL'][checked_url_2['Address'].isin(address_url_3) == True]
#             for url in redirect_url_3:
#                 redirect_url_3_address = checked_url_2[url == checked_url_2['Address']]
#                 if redirect_url_3_address['Status Code'].item() == 200:
#                     print('ok')
#                 else:
#                     address_url_4.append(url)
#             if any(checked_url_2['Address'].isin(address_url_4) == True):
#                 redirect_url_4 = checked_url_2['Redirect URL'][checked_url_2['Address'].isin(address_url_4) == True]
#                 for url in redirect_url_4:
#                     redirect_url_4_address = checked_url_2[url == checked_url_2['Address']]
#                     if redirect_url_4_address['Status Code'].item() == 200:
#                         print('ok')
#                     else:
#                         address_url_5.append(url)
#             else:
#                 break
#         else:
#             break         
#     else:
#         break
# else:
#     break

SyntaxError: 'break' outside loop (<ipython-input-4-d9083f194a30>, line 42)

## 1. Title DataFrame

In [29]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    internal_df = pd.read_excel(raw_address+'/'+country+'/Raw/internal_all.xlsx', sheet_name='검수 URL')
    
    # 오름차순 정룔
    internal_df = internal_df.sort_values(by='Address', ascending=True)

    # 컬럼 불러오기
    title_df = internal_df[['Address', 'Title 1', 'Title 1 Length']]

    # 오름차순 정렬
    title_df = title_df.sort_values(by='Address', ascending=True)

    # Exist 컬럼
    title_df['Exist'] = title_df['Title 1']
    
    title_df['Exist'].loc[title_df['Exist'].isnull() == False] = 'O'
    title_df['Exist'].loc[title_df['Exist'].isnull() == True] = 'X'

    # Length Suitability 컬럼
    title_df['Length Suitability'] = title_df['Title 1 Length']

    title_df['Length Suitability'].loc[title_df['Length Suitability'] < 29] = 0
    title_df['Length Suitability'].loc[title_df['Length Suitability'] > 29] = 1

    title_df['Length Suitability'].replace(0, 'X', inplace=True)
    title_df['Length Suitability'].replace(1, 'O', inplace=True)
    
    # Whether it is an adequate scope or not 컬럼
    title_df['Whether it is an adequate scope or not'] = title_df['Title 1 Length']

    title_df['Whether it is an adequate scope or not'][title_df['Whether it is an adequate scope or not'] == 0] = 0
    title_df['Whether it is an adequate scope or not'][(title_df['Whether it is an adequate scope or not'] > 0) &
                                                       (title_df['Whether it is an adequate scope or not'] < 30)] = 1
    title_df['Whether it is an adequate scope or not'][(title_df['Whether it is an adequate scope or not'] >= 30) &
                                                       (title_df['Whether it is an adequate scope or not'] <= 70)] = 2
    title_df['Whether it is an adequate scope or not'][title_df['Whether it is an adequate scope or not'] > 70] = 3

    title_df['Whether it is an adequate scope or not'].replace(0, 'Title Tag Not Set', inplace=True)
    title_df['Whether it is an adequate scope or not'].replace(1, 'Too Short(Under 30 Characters)', inplace=True)
    title_df['Whether it is an adequate scope or not'].replace(2, 'Suitable', inplace=True)
    title_df['Whether it is an adequate scope or not'].replace(3, 'Caution : Over 70 Characters', inplace=True)
    
    # Duplication 컬럼
    title_df['Duplication'] = title_df['Title 1']

    title_df['Duplication'][title_df['Duplication'].duplicated(keep=False) == True] = 'Duplication'
    title_df['Duplication'][title_df['Duplication'].duplicated(keep=False) == False] = ''
    
    # main 페이지가 Kia Motos Kuwait 인지 Kia Kuwait인지에 따라 결정됨 뒤에오든 앞에오든 상관 없음

    # Title Format
    title_df['Title Format'] = title_df['Title 1']

    title_df['Title Format'][title_df['Title Format'].str.contains('kia') == False] = 'Format Check Required'
    title_df['Title Format'][title_df['Title Format'].str.contains('kia') == True] = ''
    
    # 저장
    title_df.to_excel(raw_address+'/'+country+'/dataframe.xlsx', sheet_name = '1.Title', engine='openpyxl',
                      index=False)
    
    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 1. itle DataFrame 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 1. itle DataFrame 이 완료되었습니다..


## 2. Meta Description DataFrame

In [30]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    internal_df = pd.read_excel(raw_address+'/'+country+'/Raw/internal_all.xlsx', sheet_name='검수 URL')
    
    # 오름차순 정룔
    internal_df = internal_df.sort_values(by='Address', ascending=True)
    
    # 컬럼 불러오기
    desc_df = internal_df[['Address', 'Meta Description 1', 'Meta Description 1 Length']]
    
    # Exist 컬럼
    desc_df['Exist'] = desc_df['Meta Description 1']

    desc_df['Exist'].loc[desc_df['Exist'].isnull() == False] = 'O'
    desc_df['Exist'].loc[desc_df['Exist'].isnull() == True] = 'X'
    
    # Length Suitability 컬럼
    desc_df['Length Suitability'] = desc_df['Meta Description 1 Length']

    desc_df['Length Suitability'].loc[desc_df['Length Suitability'] < 99] = 0
    desc_df['Length Suitability'].loc[desc_df['Length Suitability'] > 99] = 1

    desc_df['Length Suitability'].replace(0, 'X', inplace=True)
    desc_df['Length Suitability'].replace(1, 'O', inplace=True)
    
    # Whether it is an adequate scope or not 컬럼
    desc_df['Whether it is an adequate scope or not'] = desc_df['Meta Description 1 Length']

    desc_df['Whether it is an adequate scope or not'][desc_df['Whether it is an adequate scope or not'] == 0] = 0
    desc_df['Whether it is an adequate scope or not'][(desc_df['Whether it is an adequate scope or not'] > 0) &
                                                       (desc_df['Whether it is an adequate scope or not'] < 100)] = 1
    desc_df['Whether it is an adequate scope or not'][(desc_df['Whether it is an adequate scope or not'] >= 100) &
                                                       (desc_df['Whether it is an adequate scope or not'] <= 160)] = 2
    desc_df['Whether it is an adequate scope or not'][desc_df['Whether it is an adequate scope or not'] > 160] = 3

    desc_df['Whether it is an adequate scope or not'].replace(0, 'Meta Description Not Set', inplace=True)
    desc_df['Whether it is an adequate scope or not'].replace(1, 'Too Short(Under 100 Characters)', inplace=True)
    desc_df['Whether it is an adequate scope or not'].replace(2, 'Suitable', inplace=True)
    desc_df['Whether it is an adequate scope or not'].replace(3, 'Caution: Over 160 Chacracters', inplace=True)
    
    # Duplication 컬럼
    desc_df['Duplication'] = desc_df['Meta Description 1']

    desc_df['Duplication'][desc_df['Duplication'].duplicated(keep=False) == True] = 'Duplication'
    desc_df['Duplication'][desc_df['Duplication'].duplicated(keep=False) == False] = ''

    # 저장
    with pd.ExcelWriter(raw_address+'/'+country+'/dataframe.xlsx', mode='a', engine='openpyxl') as writer:
        desc_df.to_excel(writer, sheet_name = '2. Meta Description', index=False)

    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 2. Meta Description DataFrame 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 2. Meta Description DataFrame 이 완료되었습니다..


## 3. URL & Canonical URL

In [31]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    internal_df = pd.read_excel(raw_address+'/'+country+'/Raw/internal_all.xlsx', sheet_name='검수 URL')
    
    # 오름차순 정룔
    internal_df = internal_df.sort_values(by='Address', ascending=True)

    # 컬럼 불러오기
    cano_df = internal_df[['Address', 'Canonical Link Element 1']]

    # URL Format O/X 컬럼
    cano_df['URL Format O/X'] = ''
    s_char='=+,#?:^$.@*"※~&%ㆍ!』\‘|()[]<>`…》_'
    for i in range(cano_df.shape[0]):
        url_pre = cano_df['Address'][i]
        url_pre_1 = url_pre.replace("/","")
        url_pre_2 = url_pre_1.replace("-","")
        url_pre_3 = url_pre_2.replace(".","")
        url_pre_4 = url_pre_3.replace(":","")
        if any(url_string in url_pre_4 for url_string in s_char):
            cano_df['URL Format O/X'][i] = 'X'
        else:
            cano_df['URL Format O/X'][i] = 'O'
            
    # URL : Whether it is correct or not 컬럼
    cano_df['URL : Whether it is correct or not'] = cano_df['URL Format O/X']

    cano_df['URL : Whether it is correct or not'][cano_df['URL : Whether it is correct or not'] == 'X'] = 'Using Special Character'
    cano_df['URL : Whether it is correct or not'][cano_df['URL : Whether it is correct or not'] == 'O'] = 'Suitable'

    # Canonical O/X 컬럼
    cano_df['Canonical O/X'] = cano_df['Address']

    cano_df['Canonical O/X'][cano_df['Canonical O/X'] != cano_df['Canonical Link Element 1']] = 'X'
    cano_df['Canonical O/X'][cano_df['Canonical O/X'] == cano_df['Canonical Link Element 1']] = 'O'
    
    # Canonical : Whether it is correct or not 컬럼
    cano_df['Canonical : Whether it is correct or not'] = cano_df['Canonical O/X']

    cano_df['Canonical : Whether it is correct or not'][cano_df['Canonical : Whether it is correct or not'] == 'X'] = 'Using Special Character'
    cano_df['Canonical : Whether it is correct or not'][cano_df['Canonical : Whether it is correct or not'] == 'O'] = 'Suitable'
    
    # 저장
    with pd.ExcelWriter(raw_address+'/'+country+'/dataframe.xlsx', mode='a', engine='openpyxl') as writer:
        cano_df.to_excel(writer, sheet_name='3. URL & Canonical URL', index=False)
    
    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 3. URL & Canonical URL 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 3. URL & Canonical URL 이 완료되었습니다..


## 4. H1 Tag

In [32]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    h1_raw_df = pd.read_excel(raw_address+'/'+country+'/Raw/h1_all.xlsx')
    
    # 오름차순 정룔
    h1_raw_df = h1_raw_df.sort_values(by='Address', ascending=True)

    # 컬럼 불러오기
    h1_df = h1_raw_df[['Address', 'H1-1', 'H1-2', 'Occurrences']]

    # H1-1 컬럼
    h1_df['H1-1'][h1_df['H1-1'].isnull()] = ''

    # H1-2 컬럼
    h1_df['H1-2'][h1_df['H1-2'].isnull()] = ''

    # Exist 컬럼
    h1_df['Exist'] = h1_df['Occurrences']

    h1_df['Exist'][h1_df['Exist'] == 0] = 0
    h1_df['Exist'][h1_df['Exist'] > 0] = 1

    h1_df['Exist'].replace(0, 'X', inplace=True)
    h1_df['Exist'].replace(1, 'O', inplace=True)
    
    # 저장
    with pd.ExcelWriter(raw_address+'/'+country+'/dataframe.xlsx', mode='a', engine='openpyxl') as writer:
        h1_df.to_excel(writer, sheet_name='4. H1 Tag', index=False)
    
    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 4. H1 Tag 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 4. H1 Tag 이 완료되었습니다..


## 5. Breadcrumbs 컬럼

In [33]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    bread_og_df = pd.read_excel(raw_address+'/'+country+'/Raw/custom_extraction_all.xlsx')
    
    # 오름차순 정룔
    bread_og_df = bread_og_df.sort_values(by='Address', ascending=True)

    # 컬럼 불러오기
    bread_df = bread_og_df[['Address','Breadcrumbs 1', 'Breadcrumbs 2','Breadcrumbs 3', 'Breadcrumbs 4']]
    
    # Breadcrumbs Exist 컬럼
    bread_df['Breadcrumbs Exist'] = bread_og_df['Breadcrumbs 1']

    bread_df['Breadcrumbs Exist'].loc[bread_df['Breadcrumbs Exist'].isnull() == False] = 'O'
    bread_df['Breadcrumbs Exist'].loc[bread_df['Breadcrumbs Exist'].isnull() == True] = 'X'
    
    # Breadcrumbs 1 컬럼
    bread_df['Breadcrumbs 1'][bread_df['Breadcrumbs 1'].isnull()] = ''

    # Breadcrumbs 2 컬럼
    bread_df['Breadcrumbs 2'][bread_df['Breadcrumbs 2'].isnull()] = ''

    # Breadcrumbs 3 컬럼
    bread_df['Breadcrumbs 3'][bread_df['Breadcrumbs 3'].isnull()] = ''

    # Breadcrumbs 4 컬럼
    bread_df['Breadcrumbs 4'][bread_df['Breadcrumbs 4'].isnull()] = ''
    
    # 저장
    with pd.ExcelWriter(raw_address+'/'+country+'/dataframe.xlsx', mode='a', engine='openpyxl') as writer:
        bread_df.to_excel(writer, sheet_name='5. Breadcrumbs', index=False)
    
    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 5. Breadcrumbs 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 5. Breadcrumbs 이 완료되었습니다..


## 6. OG Tag

In [34]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    bread_og_df = pd.read_excel(raw_address+'/'+country+'/Raw/custom_extraction_all.xlsx')
    
    # 오름차순 정룔
    bread_og_df = bread_og_df.sort_values(by='Address', ascending=True)

    # 컬럼 불러오기
    og_df = bread_og_df[['Address','OG : Title 1', 'OG : Description 1']]
    
    # OG : Title Exist 컬럼
    og_df['OG : Title'] = og_df['OG : Title 1'] 

    og_df['OG : Title'].loc[og_df['OG : Title'].isnull() == False] = 'O'
    og_df['OG : Title'].loc[og_df['OG : Title'].isnull() == True] = 'X'
    
    # OG : Description Exist 컬럼
    og_df['OG : Description Exist'] = og_df['OG : Description 1'] 

    og_df['OG : Description Exist'].loc[og_df['OG : Description Exist'].isnull() == False] = 'O'
    og_df['OG : Description Exist'].loc[og_df['OG : Description Exist'].isnull() == True] = 'X'
    
    # 저장
    with pd.ExcelWriter(raw_address+'/'+country+'/dataframe.xlsx', mode='a', engine='openpyxl') as writer:
        og_df.to_excel(writer, sheet_name='6. OG Tag', index=False)
    
    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 6. OG Tag 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 6. OG Tag 이 완료되었습니다..


## 7. Response Code

In [35]:
count_num = 0

for country, as_country in zip(countrys, as_countrys):
    # 데이터 불러오기
    internal_df = pd.read_excel(raw_address+'/'+country+'/Raw/internal_all.xlsx', sheet_name='1 - All')

    res_raw_df = internal_df
    res_df = res_raw_df[res_raw_df['Address'].str.contains('www.kia.com/'+as_country)]
    res_df.sort_values(by='Address', ascending=False, inplace=True)
    
    # 저장
    with pd.ExcelWriter(raw_address+'/'+country+'/dataframe.xlsx', mode='a', engine='openpyxl') as writer:
        res_df.to_excel(writer, sheet_name='7. Response Code', index=False)
        
    # 결과
    count_num += 1
    print(count_num,'. {} 나라의 7. Response Code 이 완료되었습니다..'.format(country))

1 . Pakistan(PK) 나라의 7. Response Code 이 완료되었습니다..


In [ ]:
# 평균 Bounce rate